In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import torch.utils.tensorboard as tb
from Preprocessing import preprocessing
from torch.utils.data.sampler import SequentialSampler
import seaborn as sns
import matplotlib.pyplot as  plt
import os
import logging
torch.set_printoptions(profile="full")
import pandas as pd

In [2]:
#static parameters
train_batch_size = 170
val_batch_size = 170
sequence_length=50
test_batch_size = 1
input_size = 1
hidden_size = 128
num_layer = 2
output_size = 38
clip = 3

In [3]:
#get data from preprocessing.py
dataset_path = os.path.join(os.path.abspath('..'),'Dataset\\Clementi dataset\\Clementi dataset' )
network_input, network_output = preprocessing.preprocess_notes(dataset_path)
network_input, network_output = network_input.cuda(), network_output.cuda()

# print(network_input)
#print(network_output)

In [19]:
print(network_output.max())
print(network_output.min())

tensor(37, device='cuda:0')
tensor(0, device='cuda:0')


In [20]:
print(network_input.max())
print(network_input.min())

tensor(0.9737, device='cuda:0')
tensor(0., device='cuda:0')


In [6]:
network_input.shape

torch.Size([8617, 50, 1])

In [7]:
'''
data is highly unbalanced
# '''
# sns.distplot(torch.tensor(network_output).cpu())
# xx = pd.DataFrame(torch.tensor(network_output).cpu())
# xx.groupby(0).size().to_frame(name='values')

'\ndata is highly unbalanced\n# '

In [8]:
'''
to make batch of equal sizes
Quick Fix
'''
network_input = network_input[: -117]
network_output = network_output[: -117]

print(network_input.shape)
print(network_output.shape)

torch.Size([8500, 50, 1])
torch.Size([8500])


In [9]:
'''
create Stacked LSTM model
'''
class Stacked_LSTM(nn.Module):
    def __init__(self,input_size, hidden_size, num_layers, output_size):
        super().__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size

        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, batch_first = True, dropout = 0.5)
        
        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(hidden_size, output_size)
        
        
    def forward(self, x, hidden,batch_size):
        
        output, hidden = self.lstm(x, hidden)        
        #print(output.shape)
        
        # stack up lstm outputs
        output = output.contiguous().view(-1, self.hidden_size)
        #print(output.shape)
        
        output = self.dropout(output)
        output = self.linear(output)
        #print('Linear Output :-',output.shape)
        
        #output = F.softmax(output, dim = 1)
        #print('SOFTMAX OUTPUT :--', output)
        
        
        # reshape to be batch_size first
        output = output.view(batch_size, -1)
        #print('Reshape to batch size first :-',output.shape)
        
        output = output[:, -self.output_size:] # get last batch of labels
        #print('Final Output :-',output)
        #print('RESHAPE SIZE :-', output.shape)
        
        return output, hidden
    
    def hidden_init(self,batch_size):
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.num_layers, batch_size, self.hidden_size).zero_(),
          weight.new(self.num_layers, batch_size, self.hidden_size).zero_())
        return hidden

#initialize the weights of LSTM using Xavier initialization    
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)

    

In [10]:
'''
Divide the dataset into train/val 
'''
train_size = 0.8
indices = list(range(len(network_input)))
split = int(np.floor(train_size*len(network_input)))
train_idx, val_idx = indices[:split], indices[split:]

train_sampler = SequentialSampler(train_idx)
val_sampler = SequentialSampler(val_idx)

dataset = TensorDataset(network_input,network_output)
train_loader = DataLoader(dataset, batch_size= train_batch_size, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size= val_batch_size,sampler= val_sampler)

In [15]:
import torch.optim as optimizer

model = Stacked_LSTM(input_size,hidden_size,num_layer,output_size)
model.apply(init_weights)

criterion = nn.CrossEntropyLoss()
optimizer = optimizer.AdamW(model.parameters())
#optimizer = optimizer.RMSprop(model.parameters(), lr = 0.001, weight_decay = 0.01)


#make sure to transfer model to GPU after initializing optimizer
model.cuda()

Stacked_LSTM(
  (lstm): LSTM(1, 128, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=128, out_features=38, bias=True)
)

In [16]:
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0.01
)

In [17]:
epochs = 500
min_val_loss = np.Inf

for e in range(epochs):
    
    train_loss = 0
    val_loss = 0
    train_accuracy = 0
    val_accuracy = 0
    
    
    hidden = model.hidden_init(train_batch_size)    
    #print('hidden[0].shape:- ',hidden[0].shape)
    
    for inputs,labels in train_loader:
        #print(inputs.shape)

        '''
        Creating new variables for the hidden state, otherwise
        we'd backprop through the entire training history
        '''
        h = tuple([each.data for each in hidden])
        

        # zero accumulated gradients
        model.zero_grad()
       
        # get the output from the model
        output, h = model.forward(inputs, h,train_batch_size)
        #print('OUTPUT', output)
        
        
        #print('Labels Shape :-', (torch.max(labels, 1)[1]).shape)
    
        # calculate the loss and perform backprop
        #print('Labels Long :-', labels.long())
        loss = criterion(output,labels.long())
        #print('LOSS IS :-', loss)
        
        loss.backward()
        
        #calculate training accuracy
        output = F.softmax(output, dim = 1)
        top_p, top_class = output.topk(1, dim=1)
        logging.debug(' top probab {} top class {}'.format(top_p.view(-1, top_p.shape[0]), top_class.view(-1, top_p.shape[0])))

        equals = top_class == labels.long().view(*top_class.shape)
        train_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        #nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        train_loss += loss.item()
        #print(train_loss)
              
    model.eval()
    for inputs, labels in val_loader:
                
        val_h = tuple([each.data for each in hidden])
        
        output, hidden = model.forward(inputs, val_h,val_batch_size)
       
        loss = criterion(output,labels.long())
        
        val_loss += loss.item()
        
        #calculate validation accuracy
        output = F.softmax(output, dim = 1)
        top_p, top_class = output.topk(1, dim=1)
        
        #logging.debug(output)
        #logging.debug('VALIDATION top probab {} VALIDATION top class {}'.format(top_p.view(-1, top_p.shape[0]), top_class.view(-1, top_p.shape[0])))

        #print('Top Class:- ',top_class)
        equals = top_class == labels.long().view(*top_class.shape)
        #print('Equals:- ', equals)
        val_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
    model.train()
    
    #Averaging losses
    train_loss = train_loss/len(train_loader)
    val_loss = val_loss/len(val_loader)
    val_accuracy = val_accuracy/len(val_loader)
    train_accuracy = train_accuracy/len(train_loader)
    
    print('Epoch: {}\tTrain Loss: {:.7f} \tVal Loss:{:.7f} \tTrain Acc: {:.7}% \tVal Acc: {:.7f}%'.format(e, train_loss, val_loss, train_accuracy*100,val_accuracy*100))
    
    #saving the model if validation loss is decreased
    if val_loss <= min_val_loss:
        print('Validation Loss is decreased from {:6f} to {:6f}, so saving the model weights'.format(min_val_loss, val_loss))
        torch.save(model.state_dict(), 'lstm_state.pt')
        min_val_loss = val_loss

Epoch: 0	Train Loss: 3.3177888 	Val Loss:3.0410884 	Train Acc: 8.470588% 	Val Acc: 11.4117651%
Validation Loss is decreased from    inf to 3.041088, so saving the model weights
Epoch: 1	Train Loss: 3.1162264 	Val Loss:2.9926653 	Train Acc: 8.441177% 	Val Acc: 11.4117651%
Validation Loss is decreased from 3.041088 to 2.992665, so saving the model weights
Epoch: 2	Train Loss: 3.1024166 	Val Loss:2.9727437 	Train Acc: 8.75% 	Val Acc: 11.4117651%
Validation Loss is decreased from 2.992665 to 2.972744, so saving the model weights
Epoch: 3	Train Loss: 3.0821394 	Val Loss:2.9607683 	Train Acc: 8.779412% 	Val Acc: 11.4117651%
Validation Loss is decreased from 2.972744 to 2.960768, so saving the model weights
Epoch: 4	Train Loss: 3.0779367 	Val Loss:2.9549859 	Train Acc: 8.632353% 	Val Acc: 11.4117651%
Validation Loss is decreased from 2.960768 to 2.954986, so saving the model weights
Epoch: 5	Train Loss: 3.0637497 	Val Loss:2.9466888 	Train Acc: 9.558824% 	Val Acc: 11.4117651%
Validation Loss 

Epoch: 52	Train Loss: 2.6041932 	Val Loss:2.4579236 	Train Acc: 24.19118% 	Val Acc: 28.2352950%
Validation Loss is decreased from 2.474379 to 2.457924, so saving the model weights
Epoch: 53	Train Loss: 2.5910762 	Val Loss:2.4796409 	Train Acc: 24.02941% 	Val Acc: 28.5882361%
Epoch: 54	Train Loss: 2.5778684 	Val Loss:2.4241627 	Train Acc: 24.23529% 	Val Acc: 28.5294126%
Validation Loss is decreased from 2.457924 to 2.424163, so saving the model weights
Epoch: 55	Train Loss: 2.5656474 	Val Loss:2.4191251 	Train Acc: 24.97059% 	Val Acc: 28.3529420%
Validation Loss is decreased from 2.424163 to 2.419125, so saving the model weights
Epoch: 56	Train Loss: 2.5386767 	Val Loss:2.5292156 	Train Acc: 25.08824% 	Val Acc: 25.7058832%
Epoch: 57	Train Loss: 2.5549498 	Val Loss:2.4437256 	Train Acc: 25.17647% 	Val Acc: 26.2941185%
Epoch: 58	Train Loss: 2.5364429 	Val Loss:2.3769479 	Train Acc: 25.60294% 	Val Acc: 29.5294125%
Validation Loss is decreased from 2.419125 to 2.376948, so saving the model 

KeyboardInterrupt: 

#### Music Genaration


In [ ]:
#load population database
testing_data = np.random.rand(1000)
testing_data = testing_data.reshape(testing_data.shape[0],1)
initial_seq = np.random.rand(49,1)
initial_seq = initial_seq.reshape(starting_notes.shape[0],1)

In [ ]:
#Predicting
def prediction_with_influence(influence,int2note):
    
    predicted_notes = []
    test_seq = initial_seq
    for i in range(len(testing_data)):
        test_seq.append(testing_data[i])
        test_seq = test_seq[-sequence_length:]

        test_hidden = model.hidden_init(test_batch_size)
        test_output,_ = model.forward(test_seq, test_hidden, test_batch_size)

        top_p, top_class = test_output.topk(1,dim =1)
        test_seq[sequence_length-1] = int2note[top_class]/max_note
        predicted_notes.append(int2note[top_class]/max_note)
        
        return predicted_notes